In [1]:
import numpy
import tensorflow as tf
import ml_dtypes
print(numpy.__version__)    # should be 1.26.4
print(tf.__version__)       # should be 2.15.0
print(ml_dtypes.__version__)# should be 0.2.0


1.26.4
2.15.0
0.2.0


In [1]:
import cv2
import numpy as np
import time
import tensorflow as tf

TFLITE_MODEL_PATH = 'trained_models/mobilenetv2_skin_tone.tflite'  # Update if your model is in a different path
CLASS_NAMES = ['black', 'brown', 'white']

# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']  # e.g., [1, 3, 224, 224] or [1, 224, 224, 3]

# Preprocessing function (channels-first: [1, 3, 224, 224])
def preprocess(img):
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32) / 255.0
    img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
    img = img.transpose(2, 0, 1)  # Convert HWC -> CHW
    img = np.expand_dims(img, axis=0)  # [1, 3, 224, 224]
    return img

# Haar cascades
frontal_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_profileface.xml")

def forehead_coords(face):
    x, y, w, h = face
    fw = int(w * 0.5)
    fh = int(h * 0.20)
    fx = x + (w - fw) // 2
    fy = y + int(h * 0.10)
    return fx, fy, fw, fh

cap = cv2.VideoCapture(0)
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    elapsed = int(time.time() - start_time)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = frontal_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100,100))
    detected_type = "Frontal"

    if len(faces) == 0:
        faces = profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100,100))
        detected_type = "Profile"
    if len(faces) == 0:
        flipped = cv2.flip(gray, 1)
        faces_flip = profile_cascade.detectMultiScale(flipped, scaleFactor=1.1, minNeighbors=5, minSize=(100,100))
        if len(faces_flip) > 0:
            faces = []
            for (x, y, w, h) in faces_flip:
                x_new = gray.shape[1] - x - w
                faces.append((x_new, y, w, h))
            detected_type = "Profile (Flipped)"

    if len(faces) == 0:
        cv2.putText(frame, "No face detected", (30,40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,0,255), 2)
    else:
        faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
        (x, y, w, h) = faces[0]
        fx, fy, fw, fh = forehead_coords((x, y, w, h))
        fore_img = frame[fy:fy+fh, fx:fx+fw]

        if fore_img.shape[0] > 10 and fore_img.shape[1] > 10:
            img_rgb = cv2.cvtColor(fore_img, cv2.COLOR_BGR2RGB)
            input_tensor = preprocess(img_rgb).astype(np.float32)
            interpreter.set_tensor(input_details[0]['index'], input_tensor)
            interpreter.invoke()
            output_data = interpreter.get_tensor(output_details[0]['index'])
            pred = np.argmax(output_data)
            conf = float(tf.nn.softmax(output_data)[0][pred])
            label = CLASS_NAMES[pred]
            display_text = f"{label} ({conf*100:.1f}%)"
        else:
            display_text = "N/A"

        # Mask and annotation
        overlay = frame.copy()
        cv2.rectangle(overlay, (fx, fy), (fx+fw, fy+fh), (255, 220, 0), -1)
        frame = cv2.addWeighted(overlay, 0.3, frame, 0.7, 0)
        cv2.rectangle(frame, (fx, fy), (fx+fw, fy+fh), (255, 220, 0), 2)
        cv2.putText(frame, display_text, (fx, fy-5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 220, 0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (128, 255, 0), 1)
        cv2.putText(frame, f"{detected_type}", (x, y-25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,128,255), 2)

    cv2.putText(frame, f"{elapsed} seconds", (10, frame.shape[0]-20),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("Skin Tone Analysis (forehead/side profile)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
import os
import cv2
import numpy as np
import time

MODEL_DIR = "trained_models"
CLASS_NAMES = ['black', 'brown', 'white']

PYTORCH_MODEL_PATH = os.path.join(MODEL_DIR, "mobilenetv2_skin_tone.pth")
ONNX_MODEL_PATH = os.path.join(MODEL_DIR, "mobilenetv2_skin_tone.onnx")
TF_SAVED_MODEL_PATH = os.path.join(MODEL_DIR, "tf_saved_model")
TFLITE_MODEL_PATH = os.path.join(MODEL_DIR, "mobilenetv2_skin_tone.tflite")

selected_format = "tflite" # set to "pytorch", "onnx", "tf", or "tflite"

# If using PyTorch, you need your model definition file:
# from model_def import MobileNetV2  # example, replace with your definition

if selected_format == "pytorch":
    import torch
    from torchvision import transforms
    import model_def_file  # Replace with your model's definition file
    model = model_def_file.MobileNetV2()  # Replace with actual class name
    model.load_state_dict(torch.load(PYTORCH_MODEL_PATH, map_location="cpu"))
    model.eval()
    def preprocess(img):
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
        return transform(img).unsqueeze(0)
    def predict(input_img):
        with torch.no_grad():
            logits = model(input_img)
            conf = torch.nn.functional.softmax(logits, dim=1)
            pred = conf.argmax().item()
            return pred, conf[0][pred].item()

elif selected_format == "onnx":
    import onnxruntime as ort
    ort_session = ort.InferenceSession(ONNX_MODEL_PATH)
    def preprocess(img):
        img = cv2.resize(img, (224, 224))
        img = img.astype(np.float32) / 255.0
        img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
        img = img.transpose(2,0,1)[np.newaxis,:]
        return img
    def predict(input_img):
        output = ort_session.run(None, {ort_session.get_inputs()[0].name: input_img})[0]
        pred = np.argmax(output)
        softmax = np.exp(output) / np.exp(output).sum()
        conf = float(softmax[0][pred])
        return pred, conf

elif selected_format == "tf":
    import tensorflow as tf
    model = tf.saved_model.load(TF_SAVED_MODEL_PATH)
    infer = model.signatures["serving_default"] if "serving_default" in model.signatures else model
    # Assumes input signature will work as below; adjust if needed.
    def preprocess(img):
        img = cv2.resize(img, (224, 224))
        img = img.astype(np.float32) / 255.0
        img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
        img = img[np.newaxis, ...]
        return img
    def predict(input_img):
        inputs = tf.convert_to_tensor(input_img)
        outputs = infer(inputs)
        logits = list(outputs.values())[0] if isinstance(outputs, dict) else outputs
        conf = tf.nn.softmax(logits, axis=-1)
        pred = tf.argmax(conf, axis=-1).numpy()[0]
        return pred, float(conf[0][pred])

else:  # TFLite
    import tensorflow as tf
    interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    def preprocess(img):
        img = cv2.resize(img, (224, 224))
        img = img.astype(np.float32) / 255.0
        img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
        img = img.transpose(2, 0, 1)  # CHW
        img = np.expand_dims(img, axis=0)
        return img
    def predict(input_img):
        interpreter.set_tensor(input_details[0]['index'], input_img.astype(np.float32))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        pred = np.argmax(output_data)
        softmax = np.exp(output_data) / np.exp(output_data).sum()
        conf = float(softmax[0][pred])
        return pred, conf

# Haar cascades for face detection
frontal_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_profileface.xml")

def forehead_coords(face):
    x, y, w, h = face
    fw = int(w * 0.5)
    fh = int(h * 0.20)
    fx = x + (w - fw) // 2
    fy = y + int(h * 0.10)
    return fx, fy, fw, fh

cap = cv2.VideoCapture(0)
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    elapsed = int(time.time() - start_time)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = frontal_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
    detected_type = "Frontal"

    if len(faces) == 0:
        faces = profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
        detected_type = "Profile"
    if len(faces) == 0:
        flipped = cv2.flip(gray, 1)
        faces_flip = profile_cascade.detectMultiScale(flipped, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
        if len(faces_flip) > 0:
            faces = []
            for (x, y, w, h) in faces_flip:
                x_new = gray.shape[1] - x - w
                faces.append((x_new, y, w, h))
            detected_type = "Profile (Flipped)"

    if len(faces) == 0:
        cv2.putText(frame, "No face detected", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 0, 255), 2)
    else:
        faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
        (x, y, w, h) = faces[0]
        fx, fy, fw, fh = forehead_coords((x, y, w, h))
        fore_img = frame[fy:fy+fh, fx:fx+fw]

        if fore_img.shape[0] > 10 and fore_img.shape[1] > 10:
            img_rgb = cv2.cvtColor(fore_img, cv2.COLOR_BGR2RGB)
            if selected_format == "pytorch":
                input_tensor = preprocess(img_rgb)
                pred, conf = predict(input_tensor)
            elif selected_format == "onnx":
                input_tensor = preprocess(img_rgb)
                pred, conf = predict(input_tensor)
            elif selected_format == "tf":
                input_tensor = preprocess(img_rgb)
                pred, conf = predict(input_tensor)
            else:  # TFLite
                input_tensor = preprocess(img_rgb)
                pred, conf = predict(input_tensor)
            label = CLASS_NAMES[pred]
            display_text = f"{label} ({conf*100:.1f}%)"
        else:
            display_text = "N/A"

        # Overlay and annotation
        overlay = frame.copy()
        cv2.rectangle(overlay, (fx, fy), (fx+fw, fy+fh), (255, 220, 0), -1)
        frame = cv2.addWeighted(overlay, 0.3, frame, 0.7, 0)
        cv2.rectangle(frame, (fx, fy), (fx+fw, fy+fh), (255, 220, 0), 2)
        cv2.putText(frame, display_text, (fx, fy-5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 220, 0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (128, 255, 0), 1)
        cv2.putText(frame, f"{detected_type}", (x, y-25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,128,255), 2)

    cv2.putText(frame, f"{elapsed} seconds", (10, frame.shape[0]-20),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("Skin Tone Analysis (forehead/side profile)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
import os
import cv2
import numpy as np
import time

MODEL_DIR = "trained_models"
CLASS_NAMES = ['black', 'brown', 'white']

PYTORCH_MODEL_PATH = os.path.join(MODEL_DIR, "mobilenetv2_skin_tone.pth")
ONNX_MODEL_PATH = os.path.join(MODEL_DIR, "mobilenetv2_skin_tone.onnx")
TF_SAVED_MODEL_PATH = os.path.join(MODEL_DIR, "tf_saved_model")
TFLITE_MODEL_PATH = os.path.join(MODEL_DIR, "mobilenetv2_skin_tone.tflite")

selected_format = "tflite"  # Change to "pytorch", "onnx", "tf", or "tflite"

def load_model(selected_format):
    if selected_format == "pytorch":
        import torch
        from torchvision import transforms
        import model_def_file  # Replace with your PyTorch model definition file
        model = model_def_file.MobileNetV2()  # Replace with your actual class
        model.load_state_dict(torch.load(PYTORCH_MODEL_PATH, map_location="cpu"))
        model.eval()
        def preprocess(img):
            transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ])
            return transform(img).unsqueeze(0)
        def predict(input_img):
            with torch.no_grad():
                logits = model(input_img)
                conf = torch.nn.functional.softmax(logits, dim=1)
                pred = conf.argmax().item()
                return pred, conf[0][pred].item()
        return preprocess, predict

    elif selected_format == "onnx":
        import onnxruntime as ort
        ort_session = ort.InferenceSession(ONNX_MODEL_PATH)
        def preprocess(img):
            img = cv2.resize(img, (224, 224))
            img = img.astype(np.float32) / 255.0
            img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
            img = img.transpose(2, 0, 1)[np.newaxis, ...]  # NCHW
            return img
        def predict(input_img):
            output = ort_session.run(None, {ort_session.get_inputs()[0].name: input_img})[0]
            pred = np.argmax(output)
            softmax = np.exp(output) / np.exp(output).sum()
            conf = float(softmax[0][pred])
            return pred, conf
        return preprocess, predict

    elif selected_format == "tf":
        import tensorflow as tf
        model = tf.saved_model.load(TF_SAVED_MODEL_PATH)
        infer = model.signatures["serving_default"] if "serving_default" in model.signatures else model
        def preprocess(img):
            img = cv2.resize(img, (224, 224))
            img = img.astype(np.float32) / 255.0
            img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
            img = img[np.newaxis, ...]  # NHWC
            return img
        def predict(input_img):
            import tensorflow as tf
            inputs = tf.convert_to_tensor(input_img)
            outputs = infer(inputs)
            logits = list(outputs.values())[0] if isinstance(outputs, dict) else outputs
            conf = tf.nn.softmax(logits, axis=-1)
            pred = tf.argmax(conf, axis=-1).numpy()[0]
            return pred, float(conf[0][pred])
        return preprocess, predict

    else:  # tflite
        import tensorflow as tf
        interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        def preprocess(img):
            img = cv2.resize(img, (224, 224))
            img = img.astype(np.float32) / 255.0
            img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
            # Adjust to model input: CHW or HWC (adjust if needed for your export!)
            img = img.transpose(2, 0, 1)  # NCHW
            img = np.expand_dims(img, axis=0)
            return img
        def predict(input_img):
            interpreter.set_tensor(input_details[0]['index'], input_img.astype(np.float32))
            interpreter.invoke()
            output_data = interpreter.get_tensor(output_details[0]['index'])
            pred = np.argmax(output_data)
            softmax = np.exp(output_data) / np.exp(output_data).sum()
            conf = float(softmax[0][pred])
            return pred, conf
        return preprocess, predict

# Initialize model functions
preprocess, predict = load_model(selected_format)

# Haar cascades for face detection
frontal_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_profileface.xml")

def forehead_coords(face):
    x, y, w, h = face
    fw = int(w * 0.5)
    fh = int(h * 0.20)
    fx = x + (w - fw) // 2
    fy = y + int(h * 0.10)
    return fx, fy, fw, fh

cap = cv2.VideoCapture(0)
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    elapsed = int(time.time() - start_time)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = frontal_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
    detected_type = "Frontal"

    if len(faces) == 0:
        faces = profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
        detected_type = "Profile"
    if len(faces) == 0:
        flipped = cv2.flip(gray, 1)
        faces_flip = profile_cascade.detectMultiScale(flipped, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
        if len(faces_flip) > 0:
            faces = []
            for (x, y, w, h) in faces_flip:
                x_new = gray.shape[1] - x - w
                faces.append((x_new, y, w, h))
            detected_type = "Profile (Flipped)"
    if len(faces) == 0:
        cv2.putText(frame, "No face detected", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 0, 255), 2)
    else:
        faces = sorted(faces, key=lambda x: x[2] * x[3], reverse=True)
        (x, y, w, h) = faces[0]
        fx, fy, fw, fh = forehead_coords((x, y, w, h))
        fore_img = frame[fy:fy+fh, fx:fx+fw]

        if fore_img.shape[0] > 10 and fore_img.shape[1] > 10:
            img_rgb = cv2.cvtColor(fore_img, cv2.COLOR_BGR2RGB)
            input_tensor = preprocess(img_rgb)
            pred, conf = predict(input_tensor)
            label = CLASS_NAMES[pred]
            display_text = f"{label} ({conf * 100:.1f}%)"
        else:
            display_text = "N/A"

        # Overlay/annotation
        overlay = frame.copy()
        cv2.rectangle(overlay, (fx, fy), (fx+fw, fy+fh), (255, 220, 0), -1)
        frame = cv2.addWeighted(overlay, 0.3, frame, 0.7, 0)
        cv2.rectangle(frame, (fx, fy), (fx+fw, fy+fh), (255, 220, 0), 2)
        cv2.putText(frame, display_text, (fx, fy-5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 220, 0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (128, 255, 0), 1)
        cv2.putText(frame, f"{detected_type}", (x, y-25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 128, 255), 2)

    cv2.putText(frame, f"{elapsed} seconds", (10, frame.shape[0]-20),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Skin Tone Analysis (forehead/side profile)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'model_def_file'

In [5]:
import os
import cv2
import numpy as np
import time

MODEL_DIR = "trained_models"
CLASS_NAMES = ['black', 'brown', 'white']
PYTORCH_MODEL_PATH = os.path.join(MODEL_DIR, "mobilenetv2_skin_tone.pth")

def load_model():
    import torch
    from torchvision.models import mobilenet_v2
    from torchvision import transforms

    model = mobilenet_v2(num_classes=3)
    model.load_state_dict(torch.load(PYTORCH_MODEL_PATH, map_location="cpu"))
    model.eval()
    def preprocess(img):
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
        return transform(img).unsqueeze(0)
    def predict(input_img):
        with torch.no_grad():
            logits = model(input_img)
            conf = torch.nn.functional.softmax(logits, dim=1)
            pred = conf.argmax().item()
            return pred, conf[0][pred].item()
    return preprocess, predict

preprocess, predict = load_model()
frontal_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_profileface.xml")

def forehead_coords(face):
    x, y, w, h = face
    fw = int(w * 0.5)
    fh = int(h * 0.20)
    fx = x + (w - fw) // 2
    fy = y + int(h * 0.10)
    return fx, fy, fw, fh

cap = cv2.VideoCapture(0)
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    elapsed = int(time.time() - start_time)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = frontal_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
    detected_type = "Frontal"
    if len(faces) == 0:
        faces = profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
        detected_type = "Profile"
    if len(faces) == 0:
        flipped = cv2.flip(gray, 1)
        faces_flip = profile_cascade.detectMultiScale(flipped, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
        if len(faces_flip) > 0:
            faces = []
            for (x, y, w, h) in faces_flip:
                x_new = gray.shape[1] - x - w
                faces.append((x_new, y, w, h))
            detected_type = "Profile (Flipped)"
    if len(faces) == 0:
        cv2.putText(frame, "No face detected", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 0, 255), 2)
    else:
        faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
        (x, y, w, h) = faces[0]
        fx, fy, fw, fh = forehead_coords((x, y, w, h))
        fore_img = frame[fy:fy+fh, fx:fx+fw]
        if fore_img.shape[0] > 10 and fore_img.shape[1] > 10:
            img_rgb = cv2.cvtColor(fore_img, cv2.COLOR_BGR2RGB)
            input_tensor = preprocess(img_rgb)
            pred, conf = predict(input_tensor)
            label = CLASS_NAMES[pred]
            display_text = f"{label} ({conf * 100:.1f}%)"
        else:
            display_text = "N/A"
        overlay = frame.copy()
        cv2.rectangle(overlay, (fx, fy), (fx+fw, fy+fh), (255, 220, 0), -1)
        frame = cv2.addWeighted(overlay, 0.3, frame, 0.7, 0)
        cv2.rectangle(frame, (fx, fy), (fx+fw, fy+fh), (255, 220, 0), 2)
        cv2.putText(frame, display_text, (fx, fy-5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,220,0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (128,255,0), 1)
        cv2.putText(frame, f"{detected_type}", (x, y-25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,128,255), 2)
    cv2.putText(frame, f"{elapsed} seconds", (10, frame.shape[0]-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("Skin Tone Analysis (forehead/side profile)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [11]:
import os
import cv2
import numpy as np
import time

MODEL_DIR = "trained_models"
CLASS_NAMES = ['black', 'brown', 'white']
TF_SAVED_MODEL_PATH = os.path.join(MODEL_DIR, "tf_saved_model")

def load_model():
    import tensorflow as tf
    model = tf.saved_model.load(TF_SAVED_MODEL_PATH)
    infer = model.signatures["serving_default"] if "serving_default" in model.signatures else model
    def preprocess(img):
        img = cv2.resize(img, (224, 224))
        img = img.astype(np.float32) / 255.0
        img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
        img = img[np.newaxis, ...]
        return img
    def predict(input_img):
        import tensorflow as tf
        inputs = tf.convert_to_tensor(input_img)
        outputs = infer(inputs)
        logits = list(outputs.values())[0] if isinstance(outputs, dict) else outputs
        conf = tf.nn.softmax(logits, axis=-1)
        pred = tf.argmax(conf, axis=-1).numpy()[0]
        return pred, float(conf[0][pred])
    return preprocess, predict

# The rest of the script (face detection, loop, etc.)—copy from the PyTorch example above.
preprocess, predict = load_model()
frontal_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_profileface.xml")
def forehead_coords(face):
    x, y, w, h = face
    fw = int(w * 0.5)
    fh = int(h * 0.20)
    fx = x + (w - fw) // 2
    fy = y + int(h * 0.10)
    return fx, fy, fw, fh
cap = cv2.VideoCapture(0)
start_time = time.time()
while True:
    ret, frame = cap.read()
    if not ret:
        break
    elapsed = int(time.time() - start_time)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = frontal_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
    detected_type = "Frontal"
    if len(faces) == 0:
        faces = profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
        detected_type = "Profile"
    if len(faces) == 0:
        flipped = cv2.flip(gray, 1)
        faces_flip = profile_cascade.detectMultiScale(flipped, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
        if len(faces_flip) > 0:
            faces = []
            for (x, y, w, h) in faces_flip:
                x_new = gray.shape[1] - x - w
                faces.append((x_new, y, w, h))
            detected_type = "Profile (Flipped)"
    if len(faces) == 0:
        cv2.putText(frame, "No face detected", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 0, 255), 2)
    else:
        faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
        (x, y, w, h) = faces[0]
        fx, fy, fw, fh = forehead_coords((x, y, w, h))
        fore_img = frame[fy:fy+fh, fx:fx+fw]
        if fore_img.shape[0] > 10 and fore_img.shape[1] > 10:
            img_rgb = cv2.cvtColor(fore_img, cv2.COLOR_BGR2RGB)
            input_tensor = preprocess(img_rgb)
            pred, conf = predict(input_tensor)
            label = CLASS_NAMES[pred]
            display_text = f"{label} ({conf * 100:.1f}%)"
        else:
            display_text = "N/A"
        overlay = frame.copy()
        cv2.rectangle(overlay, (fx, fy), (fx+fw, fy+fh), (255, 220, 0), -1)
        frame = cv2.addWeighted(overlay, 0.3, frame, 0.7, 0)
        cv2.rectangle(frame, (fx, fy), (fx+fw, fy+fh), (255, 220, 0), 2)
        cv2.putText(frame, display_text, (fx, fy-5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,220,0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (128,255,0), 1)
        cv2.putText(frame, f"{detected_type}", (x, y-25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,128,255), 2)
    cv2.putText(frame, f"{elapsed} seconds", (10, frame.shape[0]-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("Skin Tone Analysis (forehead/side profile)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


InvalidArgumentError: cannot compute __inference_signature_wrapper_2241 as input #0(zero-based) was expected to be a float tensor but is a double tensor [Op:__inference_signature_wrapper_2241]